<a href="https://colab.research.google.com/github/dev-S-t/Fraud_Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

> # **DATA INFO**

#### The data is broken into **two files identity and transaction,** which are **joined by TransactionID**. Not all transactions have corresponding identity information.
https://www.kaggle.com/c/ieee-fraud-detection/data?select=test_transaction.csv




>> ### **Categorical Features - Transaction**



>>>* ProductCD
* card1 - card6
* addr1, addr2
* P_emaildomain
* R_emaildomain
* M1 - M9


>> ### **Categorical Features - Identity**


>>> * DeviceType
* DeviceInfo
* id_12 - id_38



> ### **Transaction Table**

>> * TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
* TransactionAMT: transaction payment amount in USD
* ProductCD: product code, the product for each transaction
* card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
* addr: address
* dist: distance
* P_ and (R__) emaildomain: purchaser and recipient email domain
* C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
* D1-D15: timedelta, such as days between previous transaction, etc.
* M1-M9: match, such as names on card and address, etc.
* Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.


> ### **Identity Table**


>> * Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
* They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)



> # Loading Data

In [3]:
# sample_df = pd.read_csv("/content/drive/MyDrive/ieee-fraud-detection/sample_submission.csv")
test_id = pd.read_csv("/content/drive/MyDrive/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("/content/drive/MyDrive/ieee-fraud-detection/test_transaction.csv")
train_id = pd.read_csv("/content/drive/MyDrive/ieee-fraud-detection/train_identity.csv")
train_transaction = pd.read_csv("/content/drive/MyDrive/ieee-fraud-detection/train_transaction.csv")

>

# **Preprocessing**

**Merege Id and Transaction to df**

In [4]:
merge_column = 'TransactionID'
df = train_id.merge(train_transaction, how='left', on=merge_column, indicator=True)

df.loc[df['_merge'] == 'left_only', train_transaction.columns] = np.nan

# # Drop the temporary merge indicator column if not needed
# train_merged.drop('_merge', axis=1, inplace=True)

df.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,_merge
0,2987004.0,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,2987008.0,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
2,2987010.0,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,2987011.0,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,2987016.0,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both


> Exploring the data

In [5]:
df.describe()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,1.442330e+05,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,...,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000,82041.000000
mean,3.236329e+06,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,...,0.777733,723.339755,1379.108414,1017.190218,9.837929,59.213495,28.592672,55.461163,151.546395,100.950114
std,1.788496e+05,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,...,4.735065,6222.466950,11181.344838,7964.623789,244.320100,388.035252,275.048519,669.707372,1096.739466,816.354359
min,2.987004e+06,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.077142e+06,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.198818e+06,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.392923e+06,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.577534e+06,0.000000,999595.000000,10.000000,0.000000,52.000000,0.000000,61.000000,0.000000,25.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.000000,55125.000000,104060.000000,104060.000000,104060.000000


In [6]:
df.shape

(144233, 435)

In [7]:
df.isnull().sum().max()

144233

In [8]:

# Calculate percentage of null values in each column
null_percentages = (df.isnull().sum() / len(df)) * 100

# Create a bar chart with interactive zoom/pan and column names
fig = px.bar(x=df.columns, y=null_percentages, labels={'x': 'Column Name', 'y': 'Percentage of Null Values (%)'})
fig.update_layout(xaxis_title='Column Name', yaxis_title='Percentage of Null Values (%)')
fig.update_xaxes(range=[0, len(df.columns)])  # Set x-axis range for better display
fig.show()

In [9]:
result_df = pd.DataFrame(columns=["col_name", "null_count", "null_and_isFraud", "not_null_and_isFraud"])

for col in df.columns:
  # Count null values
  null_count = df[col].isnull().sum()

  # Count null values where 'isFraud' is 1
  null_and_isFraud = df[(df[col].isnull()) & (df['isFraud'] == 1)].shape[0]

  # Count not null values where 'isFraud' is 1
  not_null_and_isFraud = df[(df[col].notnull()) & (df['isFraud'] == 1)].shape[0]

  result_df = pd.concat([result_df, pd.DataFrame({"col_name": col, "null_count": null_count, "null_and_isFraud": null_and_isFraud, "not_null_and_isFraud": not_null_and_isFraud}, index=[0])], ignore_index=True)

result_df.head()


,col_name,null_count,null_and_isFraud,not_null_and_isFraud
0,TransactionID,0,0,11318
1,id_01,0,0,11318
2,id_02,3361,96,11222
3,id_03,77909,4206,7112
4,id_04,77909,4206,7112


In [10]:
import pandas as pd

def analyze_columns(df):
  """
      A new pandas dataframe with four columns:
          col_name: Name of the column in the original dataframe.
          null_count: Percentage of null values in the column.
          null_and_isFraud: Percentage of rows with null value and "isFraud" as 1.
          not_null_and_isFraud: Percentage of rows with non-null value and "isFraud" as 1.
  """

  result_df = pd.DataFrame(columns=["col_name", "null_count", "null_and_isFraud", "not_null_and_isFraud"])
  for col in df.columns:
    total_rows = len(df)
    null_count = df[col].isnull().sum() / total_rows * 100
    null_fraud_count = df[(df[col].isnull()) & (df["isFraud"] == 1)].shape[0] / total_rows * 100
    not_null_fraud_count = df[(df[col].notnull()) & (df["isFraud"] == 1)].shape[0] / total_rows * 100

    result_df = pd.concat([result_df, pd.DataFrame({
        "col_name": col,
        "null_count": null_count,
        "null_and_isFraud": null_fraud_count,
        "not_null_and_isFraud": not_null_fraud_count
    }, index=[0])], ignore_index=True)
  return result_df


result_df = analyze_columns(df.copy())


result_df.head()


,col_name,null_count,null_and_isFraud,not_null_and_isFraud
0,TransactionID,0.000000,0.000000,7.847025
1,id_01,0.000000,0.000000,7.847025
2,id_02,2.330257,0.066559,7.780466
3,id_03,54.016071,2.916115,4.930910
4,id_04,54.016071,2.916115,4.930910


In [11]:
import plotly.graph_objs as go
from plotly.offline import iplot


names = result_df['col_name']
subject1 = result_df['null_count']
subject2 = result_df['null_and_isFraud']
subject3 = result_df['not_null_and_isFraud']

# Creating the stacked bar chart
trace1 = go.Bar(
    x=names,
    y=subject1,
    name='Null'
)

trace2 = go.Bar(
    x=names,
    y=subject2,
    name='imp_null'
)

trace3 = go.Bar(
    x=names,
    y=subject3,
    name='imp_data'
)

data = [trace1, trace2, trace3]

layout = go.Layout(
    barmode='overlay',
    title='NUll Analysis',
    xaxis=dict(title='Feature Names'),
    yaxis=dict(title='Percentage')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)


In [ ]:
import pandas as pd
import plotly.graph_objects as go


# Initialize a list to hold all the bar segments
bar_segments = []

# Loop over each column except 'isFraud'
for column in df.columns:
    if column != 'isFraud':
        # Count occurrences where isFraud == 1 for each unique value in the column
        counts = df[df['isFraud'] == 1].groupby(column).size()

        # Create a bar segment for each unique value
        for value, count in counts.items():
            bar_segments.append(go.Bar(
                name=f"{column}: {value}",
                x=[column],
                y=[count],
                text=[f"{column}: {value}"],
                hoverinfo='text',
                showlegend=True
            ))

# Create the figure and add all bar segments
fig = go.Figure(data=bar_segments)

# Update layout for better visualization
fig.update_layout(
    barmode='stack',
    title='Fraud Counts by Category for Each Column',
    xaxis_title='Columns',
    yaxis_title='Count of Fraud Cases (isFraud = 1)',
    legend_title='Categories'
)

# Show the figure
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

bar_partitions_data = []

# Iterate over each column in the DataFrame except 'isFraud'.
for column in df.columns:
    if column != 'isFraud':
        # Get the unique values and their counts where isFraud is 1.
        fraud_counts = df[df['isFraud'] == 1][column].value_counts().reset_index()
        fraud_counts.columns = [column, 'count']

        # Add a column name entry to differentiate the partitions in the bar.
        fraud_counts['column_name'] = column

        # Append the data to the list.
        bar_partitions_data.append(fraud_counts)

# Concatenate all the partition data into a single DataFrame.
all_partitions_df = pd.concat(bar_partitions_data)

# Create the bar graph using Plotly Express.
fig = px.bar(all_partitions_df, x='column_name', y='count', color=column,
             title='Distribution of Fraudulent Transactions Across Columns')

# Show the figure.
fig.show()
